In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('merged_nhanes_all_years.csv')

In [3]:
data.shape

(27706, 21)

In [4]:
list(data.columns.values)

['SEQN',
 'LBXGLU',
 'LBXIN',
 'LBXSBU',
 'LBXSCR',
 'LBXSATSI',
 'SMQ020',
 'SMQ040',
 'EverDrank',
 'DrinkFrequency',
 'AvgDrinksPerDay',
 'RIDAGEYR',
 'RIAGENDR',
 'DMDEDUC2',
 'INDFMPIR',
 'DIQ010',
 'DID040',
 'DIQ050',
 'DIQ070',
 'SurveyCycle',
 'URDACT']

In [5]:
unique_SEQN_count = data['SEQN'].nunique()
print("Number of unique IDs:", unique_SEQN_count)

Number of unique IDs: 27706


In [6]:
duplicate_count = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [7]:
def create_data_dictionary(data):
    data_dict = {
        "Feature Name": [],
        "Data Type": [],
        "Missing Values": [],
        "Unique Values": [],
        "Description": [],
    }

    # Feature descriptions for your current dataset
    feature_descriptions = {
        "SEQN": "Respondent sequence number (unique ID for each participant)",
        "LBXGLU": "Fasting glucose (mg/dL)",
        "LBXIN": "Fasting insulin (μU/mL)",
        "LBXSBU": "Blood urea nitrogen (BUN) (mg/dL), marker of kidney function",
        "LBXSCR": "Serum creatinine (mg/dL), used to assess kidney function",
        "LBXSATSI": "Serum sodium concentration (mmol/L)",
        "URDACT": "Urine albumin-to-creatinine ratio (mg/g), marker of early kidney damage",
        "URXUMA": "Urine albumin (μg/mL)",
        "URXUCR": "Urine creatinine (mg/dL)",
        "SMQ020": "Ever smoked at least 100 cigarettes in life (1 = Yes, 2 = No)",
        "SMQ040": "Current smoking status (1 = Every day, 2 = Some days, 3 = Not at all)",
        "ALQ111": "Ever had at least one alcoholic drink (1 = Yes, 2 = No)",
        "ALQ121": "Drinking frequency over past 12 months",
        "ALQ130": "Average number of alcoholic drinks per day (past 12 months)",
        "RIAGENDR": "Gender (1 = Male, 2 = Female)",
        "RIDAGEYR": "Age in years at time of screening",
        "DMDEDUC2": "Education level (1 = Less than 9th grade to 5 = College graduate or above)",
        "INDFMPIR": "Ratio of family income to poverty level (higher = higher income)",
        "DIQ010": "Doctor told you have diabetes (1 = Yes, 2 = No)",
        "DID040": "Age when first told you had diabetes",
        "DIQ050": "Currently taking insulin (1 = Yes, 2 = No)",
        "DIQ070": "Currently taking pills to lower blood sugar (1 = Yes, 2 = No)",
        "SurveyCycle": "NHANES survey cycle years"
    }

    # Loop through each column in the DataFrame
    for column in data.columns:
        data_dict["Feature Name"].append(column)
        data_dict["Data Type"].append(data[column].dtype)
        data_dict["Missing Values"].append(data[column].isnull().sum())
        data_dict["Unique Values"].append(data[column].nunique())
        data_dict["Description"].append(feature_descriptions.get(column, "No description available"))

    return pd.DataFrame(data_dict)
# Optional: remove a column if it exists
data = data.drop(columns=['Calculated_Diff'], errors='ignore')

# Create and view the data dictionary
data_dict_df = create_data_dictionary(data)
data_dict_df

,Feature Name,Data Type,Missing Values,Unique Values,Description
0,SEQN,float64,0,27706,Respondent sequence number (unique ID for each...
1,LBXGLU,float64,1489,1332,Fasting glucose (mg/dL)
2,LBXIN,float64,2015,4210,Fasting insulin (μU/mL)
3,LBXSBU,float64,1801,75,"Blood urea nitrogen (BUN) (mg/dL), marker of k..."
4,LBXSCR,float64,1800,317,"Serum creatinine (mg/dL), used to assess kidne..."
5,LBXSATSI,float64,1826,210,Serum sodium concentration (mmol/L)
6,SMQ020,float64,140,4,Ever smoked at least 100 cigarettes in life (1...
7,SMQ040,float64,15294,3,"Current smoking status (1 = Every day, 2 = Som..."
8,EverDrank,float64,17061,3,No description available
9,DrinkFrequency,float64,6055,82,No description available


In [8]:
data.info()
data.describe()

missing_values = data.isnull().sum()
missing_percent = (missing_values / len(data)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing %': missing_percent.round(2)
})
print("\nMissing data summary:")
print(missing_df[missing_df['Missing Values'] > 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27706 entries, 0 to 27705
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEQN             27706 non-null  float64
 1   LBXGLU           26217 non-null  float64
 2   LBXIN            25691 non-null  float64
 3   LBXSBU           25905 non-null  float64
 4   LBXSCR           25906 non-null  float64
 5   LBXSATSI         25880 non-null  float64
 6   SMQ020           27566 non-null  float64
 7   SMQ040           12412 non-null  float64
 8   EverDrank        10645 non-null  float64
 9   DrinkFrequency   21651 non-null  float64
 10  AvgDrinksPerDay  16688 non-null  float64
 11  RIDAGEYR         27706 non-null  float64
 12  RIAGENDR         27706 non-null  float64
 13  DMDEDUC2         27078 non-null  float64
 14  INDFMPIR         25038 non-null  float64
 15  DIQ010           27706 non-null  float64
 16  DID040           3383 non-null   float64
 17  DIQ050      

In [11]:
data = data.rename(columns={
    'DIQ010': 'HasDiabetes',
    'DID040': 'AgeAtDiagnosis',
    'DIQ050': 'DiagnosedByDoctor',
    'DIQ070': 'StartedInsulin'
})

# Step 1: Define a function to flag likely Type 1 Diabetes cases
def is_likely_type1(row):
    return (
        row['HasDiabetes'] == 1 and
        pd.notnull(row['AgeAtDiagnosis']) and row['AgeAtDiagnosis'] < 30 and
        row['DiagnosedByDoctor'] == 1 and
        (row['StartedInsulin'] == 2 or pd.isnull(row['StartedInsulin']))
    )

# Step 2: Apply the function to create a flag
data['LikelyType1'] = data.apply(is_likely_type1, axis=1)

# Step 3: Define Type 2 Diabetes as diagnosed diabetes that is not Type 1
data['HasType2Diabetes'] = data['HasDiabetes'].apply(lambda x: 1 if x == 1 else 0)
data.loc[data['LikelyType1'], 'HasType2Diabetes'] = 0  # Remove type 1s from type 2 flag

# Preview results
print(data[['HasDiabetes', 'LikelyType1', 'HasType2Diabetes']].value_counts())

HasDiabetes  LikelyType1  HasType2Diabetes
2.0          False        0                   23753
1.0          False        1                    3212
3.0          False        0                     554
1.0          True         0                     173
9.0          False        0                      14
dtype: int64
